# Lecture 19 – Data 100, Summer 2024

Data 100, Summer 2024

[Acknowledgments Page](https://ds100.org/su24/acks/)

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import sklearn.linear_model as lm
import seaborn as sns


---

## Simple Bootstrap Example

Here we work through a simple example of the bootstap when estimating the relationship between miles per gallon and the weight of a vehicle.

Suppose we collected a sample of 20 cars from a population.  For the purposes of this demo we will assume that the seaborn dataset is the population. The following is a visualization of our sample:

In [2]:
np.random.seed(42)
mpg_sample = sns.load_dataset('mpg').sample(20)
px.scatter(mpg_sample, x='weight', y='mpg', trendline='ols', width=800)

Fitting a linear model we get an estimate of the slope:

In [3]:
model = lm.LinearRegression().fit(mpg_sample[['weight']], mpg_sample['mpg'])
model.coef_

array([-0.00692182])

### Bootstrap Implementation

Now let's use bootstrap to estimate the distribution of that coefficient.  Here will will construct a bootstrap function that takes an estimator function and uses that function to construct many bootstrap estimates.

In [4]:
def estimator(sample):
    model = lm.LinearRegression().fit(sample[['weight']], sample['mpg'])
    return model.coef_[0]

This code uses `df.sample` ([link](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html)) to generate a bootstrap sample of the same size of the original sample.

In [5]:
def bootstrap(sample, estimator, num_repetitions):
    """
    Returns the statistic computed on a num_repetitions  
    bootstrap samples from sample.
    """
    stats = []
    for i in np.arange(num_repetitions):
        # Step 1: Sample the Sample
        bootstrap_sample = sample.sample(frac=1, replace=True)
        # Step 2: compute statistics on the sample of the sample
        bootstrap_stat = estimator(bootstrap_sample)
        # Accumulate the statistics
        stats.append(bootstrap_stat)
    return stats    

Constructing MANY bootstrap slope estimates.

In [6]:
bs_thetas = bootstrap(mpg_sample, estimator, 10000)

We can visualize the bootstrap distribution of the slope estimates.

In [7]:
px.histogram(bs_thetas, title='Bootstrap Distribution of the Slope', 
             width=800)

### Computing a Bootstrap CI
We can compute the CI using the percentiles of the empirical distribution:

In [8]:
def bootstrap_ci(bootstrap_samples, confidence_level=95):
    """
    Returns the confidence interval for the bootstrap samples.
    """
    lower_percentile = (100 - confidence_level) / 2
    upper_percentile = 100 - lower_percentile
    return np.percentile(bootstrap_samples, [lower_percentile, upper_percentile])

In [9]:
bootstrap_ci(bs_thetas)

array([-0.00861209, -0.0055226 ])

### Comparing to the Population CIs 
In practice you don't have access to the population but in this specific example we had taken a sample from a larger dataset that we can pretend is the population.  Let's compare to resampling from the larger dataset:

In [10]:
mpg_pop = sns.load_dataset('mpg')
theta_est = [estimator(mpg_pop.sample(20)) for i in range(10000)]

In [11]:
bootstrap_ci(theta_est)

array([-0.01017473, -0.00574973])

Visualizing the two distributions:

In [12]:
thetas = pd.DataFrame({"bs_thetas": bs_thetas, "thetas": theta_est})
px.histogram(thetas.melt(), x='value', facet_row='variable', 
             title='Distribution of the Slope', width=800)

Back to lecture!
<br><br>

---

## PurpleAir

This example is from the Data 100 textbook: [link](https://learningds.org/ch/17/inf_pred_gen_boot.html). The following cell does some basic data cleaning. 

In [13]:
csv_file = 'data/Full24hrdataset.csv'
usecols = ['Date', 'ID', 'region', 'PM25FM', 'PM25cf1', 'TempC', 'RH', 'Dewpoint']
full_df = pd.read_csv(csv_file, usecols=usecols, parse_dates=['Date']).dropna()
full_df.columns = ['date', 'id', 'region', 'pm25aqs', 'pm25pa', 'temp', 'rh', 'dew']
full_df = full_df[(full_df['pm25aqs'] < 50)]
# drop dates with issues in the data
bad_dates = ['2019-08-21', '2019-08-22', '2019-09-24']
GA = full_df[(full_df['id'] == 'GA1') & (~full_df['date'].isin(bad_dates))]
GA = GA.sort_values("pm25aqs")

### Inverse Regression

After we build the model that adjusts the PurpleAir measurements using AQS, we then flip the model around and use it to predict the true air quality in the future from PurpleAir measurements when wec don't have a nearby AQS instrument. 
This is a *calibration* scenario.
Since the AQS measurements are close to the truth, we fit the more variable PurpleAir measurements to them;
this is the calibration procedure. 
Then, we use the calibration curve to correct future PurpleAir measurements. 
This two-step process is encapsulated in the simple linear model and its flipped form below.

Inverse regression:
* First, we fit a line to predict a PA measurement from the ground truth, as recorded by an AQS instrument:

    $$ \text{PA} \approx \theta_0 + \theta_1\text{AQS} $$

* Next, we flip the line around to use a PA measurement to predict the air quality,

    $$ \text{True Air Quality} \approx -\theta_0/\theta_1 + 1/\theta_1 \text{PA} $$
    
Why perform this “inverse regression”?
* Intuitively, AQS measurements are “true” and have no error.
* A linear model takes a “true” x value input and minimizes the error in the y direction.
* Algebraically identical, but statistically different.


In [14]:
AQS, PA = GA[['pm25aqs']], GA['pm25pa']

model = lm.LinearRegression().fit(AQS, PA)
theta_0, theta_1 = model.intercept_, model.coef_[0], 

In [15]:
fig = px.scatter(GA, x='pm25aqs', y='pm25pa', width=800)
xtest = np.array([GA['pm25aqs'].min(), GA['pm25aqs'].max()])
fig.add_scatter(x=xtest, y=model.predict(xtest.reshape(-1, 1)), mode='lines', 
                name="Least Squares Fit")

c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



Constructing the inverse predictions

In [16]:
print(f"True Air Quality Estimate = {-theta_0/theta_1:.2} + {1/theta_1:.2}PA") 

True Air Quality Estimate = 1.6 + 0.46PA


In [17]:
model2 = lm.LinearRegression().fit(GA[['pm25pa']], GA['pm25aqs'])

fig = px.scatter(GA, y='pm25aqs', x='pm25pa', width=800)
xtest = np.array([GA['pm25pa'].min(), GA['pm25pa'].max()])
fig.add_scatter(x=xtest, y=xtest *1/theta_1 - theta_0/theta_1 , mode='lines', 
                name="Inverse Fit")
fig.add_scatter(x=xtest, y=model2.predict(xtest.reshape(-1, 1)), mode='lines',
                name="Least Squares Fit")


c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names




### The Barkjohn et al. model with Relative Humidity

[Karoline Barkjohn, Brett Gannt, and Andrea Clements](https://amt.copernicus.org/articles/14/4617/2021/) from the US Environmental Protection Agency developed a model to improve the PuprleAir measurements from the AQS sensor measurements. arkjohn and group’s work was so successful that, as of this writing, the official US government maps, like the [AirNow Fire and Smoke](https://fire.airnow.gov/) map, includes both AQS and PurpleAir sensors, and applies Barkjohn’s correction to the PurpleAir data.
$$
\begin{aligned}
\text{PA} \approx \theta_0 + \theta_1 \text{AQS} + \theta_2 \text{RH}
\end{aligned}
$$

The model that Barkjohn settled on incorporated the **relative humidity**:

In [18]:
AQS_RH, PA = GA[['pm25aqs', 'rh']], GA['pm25pa']
model_h = lm.LinearRegression().fit(AQS_RH, PA)
[theta_1, theta_2], theta_0 = model_h.coef_, model_h.intercept_
    
print(f"True Air Quality Estimate = {-theta_0/theta_1:1.2} + {1/theta_1:.2}PA + {-theta_2/theta_1:.2}RH") 

True Air Quality Estimate = 7.0 + 0.44PA + -0.092RH


In [19]:
fig = px.scatter(GA, x='pm25aqs', y='pm25pa', width=800)
xtest = np.array([GA['pm25aqs'].min(), GA['pm25aqs'].max()])
fig.add_scatter(x=xtest, y=model.predict(xtest.reshape(-1, 1)), mode='lines', 
                name="Least Squares Fit")
fig.add_scatter(x=GA["pm25aqs"], y=model_h.predict(AQS_RH), mode='lines+markers',
                marker_size=5, name="Least Squares Fit with RH")

c:\Users\lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



**Bonus** Visualizing the surface:

In [20]:
fig = px.scatter_3d(GA, x='pm25aqs', y='rh', z='pm25pa', width=800, height=600)

grid_resolution = 2
(u,v) = np.meshgrid(
    np.linspace(GA["pm25aqs"].min(), GA["pm25aqs"].max(), grid_resolution),
    np.linspace(GA["rh"].min(), GA["rh"].max(), grid_resolution))
zs = model_h.predict(pd.DataFrame({"pm25aqs": u.flatten(), "rh": v.flatten()}))
zs_old = model.predict(pd.DataFrame({"pm25aqs": u.flatten()}))
# create the Surface
color1 = px.colors.qualitative.Plotly[3]
color2 = px.colors.qualitative.Plotly[4]
fig.add_surface(x=u, y=v, z= zs.reshape(u.shape), opacity=1, 
                colorscale=[[0, color1], [1,color1]],
                showscale=False, name="AQS + RH")
fig.add_surface(x=u, y=v, z= zs_old.reshape(u.shape), opacity=1, 
                colorscale=[[0, color2], [1,color2]],
                showscale=False, name="AQS")
# set the aspect ratio of the 3d plot
fig.update_scenes(aspectmode='cube')

<br/>Compared to the simple linear model that only incorporated AQS, the Barkjohn et al. model with relative humidity achieves lower error. Good for prediction!

<br><br> 

---


### Bootstrapping the regression coefficients for Purple Air

From the Barkjohn et al., model, AQS coefficient $\hat{\theta}_1$:

In [21]:
theta_1

2.2540167939150546

The Relative Humidity coefficient $\hat{\theta}_2$ is pretty close to zero:

In [22]:
theta_2

0.20630108775555359

Is incorporating humidity in the model really needed?

**Null hypothesis**: The null hypothesis is $\theta_2 = 0$; that is, the null model is the simpler model:

$$
\begin{aligned}
\text{PA} \approx \theta_0 + \theta_1 \text{AQS}
\end{aligned}
$$

Repeat 10,000 times to get an approximation to the boostrap sampling distirbution of the bootstrap statistic (the fitted humidity coefficient $\hat{\theta_2}$):

In [23]:
def theta2_estimate(sample):
    model = lm.LinearRegression().fit(sample[['pm25aqs', 'rh']], sample['pm25pa'])
    return model.coef_[1]

In [24]:
bs_theta2 = bootstrap(GA, theta2_estimate, 10000)

In [25]:
import plotly.express as px
px.histogram(x=bs_theta2,
            labels=dict(x='Bootstrapped Humidity Coefficient'),
            width=800)

(We know that the center will be close to the original coefficient estimated from the sample, 0.21.)

By design, the center of the bootstrap sampling distribution will be near $\hat{\theta}$ because the bootstrap population consists of the observed data. 
So, rather than compute the chance of a value at least as large as the observed statistic, we find the chance of a value at least as small as 0.

**The hypothesized value of 0 is far from the sampling distribution:**

In [26]:
len([elem for elem in bs_theta2 if elem < 0.0]) 

0

None of the 10,000 simulated regression coefficients are as small as the hypothesized coefficient. Statistical logic leads us to **reject the null hypothesis that we do not need to adjust the model for humidity.**

Back to lecture!
<br><br>

---

## The Snowy Plover

This example borrows some wording from Spring 2020's Data 100, Lecture 22.

### The Data 
The [Snowy Plover](https://www.audubon.org/field-guide/bird/snowy-plover) is a tiny bird that lives on the coast in parts of California and elsewhere. It is so small that it is vulnerable to many predators and to people and dogs that don't look where they are stepping when they go to the beach. It is considered endangered in many parts of the US.

The data are about the eggs and newly-hatched chicks of the Snowy Plover. Here's a [parent bird and some eggs](http://cescos.fau.edu/jay/eps/articles/snowyplover.html).

![plover and eggs](images/plover_eggs.jpg)

The data were collected at the Point Reyes National Seashore by a former [student at Berkeley](https://openlibrary.org/books/OL2038693M/BLSS_the_Berkeley_interactive_statistical_system). The goal was to see how the size of an egg could be used to predict the weight of the resulting chick. The bigger the newly-hatched chick, the more likely it is to survive.

![plover and chick](images/plover_chick.jpg)

Each row of the data frame below corresponds to one Snowy Plover egg and the resulting chick. Note how tiny the bird is:

* Egg Length and Egg Breadth (widest diameter) are measured in millimeters
* Egg Weight and Bird Weight are measured in grams; for comparison, a standard paper clip weighs about one gram

In [27]:
eggs = pd.read_csv('data/snowy_plover.csv')
eggs.head()

,egg_weight,egg_length,egg_breadth,bird_weight
0,7.4,28.80,21.84,5.2
1,7.7,29.04,22.45,5.4
2,7.9,29.36,22.48,5.6
3,7.5,30.10,21.71,5.3
4,8.3,30.17,22.75,5.9


In [28]:
eggs.shape

(44, 4)

For a particular egg, $x$ is the vector of length, breadth, and weight. The model is

$$
f_\theta(x) ~ = ~ \theta_0 + \theta_1\text{egg\_length} + \theta_2\text{egg\_breadth} + \theta_3\text{egg\_weight} + \epsilon
$$

- For each $i$, the parameter $\theta_i$ is a fixed number but it is unobservable. We can only estimate it.
- The random error $\epsilon$ is also unobservable, but it is assumed to have expectation 0 and be independent and identically distributed across eggs.

In [29]:
y = eggs["bird_weight"]
X = eggs[["egg_weight", "egg_length", "egg_breadth"]]
    
model = lm.LinearRegression(fit_intercept=True).fit(X, y)

display(pd.DataFrame(
    [model.intercept_] + list(model.coef_),
    columns=['theta_hat'],
    index=['intercept', 'egg_weight', 'egg_length', 'egg_breadth']))

print("RMSE", np.mean((y - model.predict(X)) ** 2))      

,theta_hat
intercept,-4.605670
egg_weight,0.431229
egg_length,0.066570
egg_breadth,0.215914


RMSE 0.04547085380275766


Let's try bootstrapping the sample to obtain a 95% confidence intervals for all the parameters.

In [30]:
def all_thetas(sample):
    # first feature
    model = lm.LinearRegression().fit(
        sample[["egg_weight", "egg_length", "egg_breadth"]],
        sample["bird_weight"])
    return [model.intercept_] + model.coef_.tolist()

In [31]:
bs_thetas = pd.DataFrame(
    bootstrap(eggs, all_thetas, 10_000), 
    columns=['intercept', 'egg_weight', 'egg_length', 'egg_breadth'])
bs_thetas

,intercept,egg_weight,egg_length,egg_breadth
0,2.735836,0.933480,-0.085565,-0.082362
1,-7.963464,0.293276,0.098463,0.372370
2,-0.571565,0.439917,0.045989,0.065429
3,-5.420348,0.319199,0.071474,0.285354
4,-9.009859,0.084618,0.113050,0.474270
...,...,...,...,...
9995,-7.254253,0.413497,0.068339,0.336043
9996,-1.593437,0.638769,0.032167,0.053375
9997,-1.076720,0.714224,-0.012315,0.064064
9998,-7.512390,0.362088,0.092687,0.333736


Computing the confidence intervals for all the coefficients we get:

In [32]:
(bs_thetas.apply(bootstrap_ci)
    .T
    .rename(columns={0: 'lower', 1: 'upper'}))

,lower,upper
intercept,-15.474294,5.035731
egg_weight,-0.265432,1.093108
egg_length,-0.097623,0.211462
egg_breadth,-0.256738,0.760643


Because all the confidence intervals contain 0 we cannot reject the null hypothesis for any of them.  Does this mean that all the parameters could be zero?


<br/><br/>

---

### Inspecting the Relationship between Features

To see what's going on, we'll make a scatter plot matrix for the data.

In [33]:
px.scatter_matrix(eggs, width=600, height=600)

This shows that `bird_weight`
is highly correlated with all the other
variables (the bottom row), which means fitting a linear model is a good idea.
But we also see that `egg_weight` is highly correlated with all the variables
(the top row).
This means we can't increase one covariate while
keeping the others constant. The individual slopes have no meaning.

Here's the correlations showing this more succinctly:

In [34]:
px.imshow(eggs.corr().round(2), text_auto=True, width=600)

<br/><br/>

---

### Changing Our Modeling Features

One way to fix this is to fit a model that only uses `egg_weight`.
This model performs almost as well as the model that uses all three variables,
and the confidence interval for $\theta_1$ doesn't 
contain zero.

In [35]:
px.scatter(eggs, x='egg_weight', y='bird_weight', trendline='ols', width=800)

In [36]:
y = eggs["bird_weight"]
X = eggs[["egg_weight"]]
    
model = lm.LinearRegression(fit_intercept=True).fit(X, y)

display(pd.DataFrame([model.intercept_] + list(model.coef_),
             columns=['theta_hat'],
             index=['intercept', 'egg_weight']))
print("RMSE", np.mean((y - model.predict(X)) ** 2))

,theta_hat
intercept,-0.058272
egg_weight,0.718515


RMSE 0.046493941375556846


In [37]:
def egg_weight_coeff(sample):
    # first feature
    model = lm.LinearRegression().fit(
        sample[["egg_weight"]],
        sample["bird_weight"])
    return [model.intercept_] + model.coef_.tolist()

In [38]:
bs_thetas_egg_weight = pd.DataFrame(
    bootstrap(eggs, egg_weight_coeff, 10_000), 
    columns=['intercept', 'egg_weight'])
bs_thetas_egg_weight

,intercept,egg_weight
0,0.190292,0.686176
1,-0.638902,0.785541
2,0.186698,0.688066
3,-0.125817,0.723262
4,0.285226,0.676833
...,...,...
9995,-0.049395,0.716097
9996,-0.024848,0.712183
9997,1.043560,0.584261
9998,-0.830942,0.815550


In [39]:
(bs_thetas_egg_weight.apply(bootstrap_ci)
    .T
    .rename(columns={0: 'lower', 1: 'upper'}))

,lower,upper
intercept,-0.904254,0.937528
egg_weight,0.601164,0.820243


It's no surprise that if you want to predict the
weight of the newly-hatched chick, using the weight
of the egg is your best move.

As this example shows, checking for collinearity is important for inference.
When we fit a model on highly correlated variables, we might not be able to
use confidence intervals to conclude that variables are
related to the prediction.